In [1]:
import tensorflow as tf
import numpy as np
import tensorflow_util as d2l

2024-04-10 15:34:02.407863: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-10 15:34:02.481023: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-10 15:34:03.638466: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### 生成数据集

In [2]:
true_w = tf.constant([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

### 读取数据集

In [3]:
def load_array(data_arrays, batch_size, is_train=True):
    """
    构造一个 Tensorflow 数据迭代器
    """
    dataset = tf.data.Dataset.from_tensor_slices(data_arrays)
    if is_train:
        dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size)
    return dataset

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [4]:
next(iter(data_iter))

(<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
 array([[ 2.9869752 ,  0.55253696],
        [ 1.0346434 , -1.0987892 ],
        [-0.3780557 ,  2.2334092 ],
        [-0.56031334, -1.0019677 ],
        [-0.0628558 ,  0.40771434],
        [ 2.0725393 ,  0.15375224],
        [-1.7860249 ,  0.48444983],
        [ 0.8200583 , -0.34010103],
        [ 0.23255555, -1.6968869 ],
        [ 0.7314172 ,  1.2821906 ]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[ 8.30052  ],
        [10.009058 ],
        [-4.164083 ],
        [ 6.5049553],
        [ 2.6908915],
        [ 7.82181  ],
        [-1.0126827],
        [ 6.9919257],
        [10.440559 ],
        [ 1.2917929]], dtype=float32)>)

### 定义模型

In [5]:
# Keras 会自动推断每个层输入的形状

net = tf.keras.Sequential()
net.add(tf.keras.layers.Dense(1))

### 初始化模型参数

In [6]:
initializer = tf.initializers.RandomNormal(stddev=0.01)
# 重新创建 net
net = tf.keras.Sequential()
# 重点：在后端执行时，初始化实际上是 推迟（deferred）执行的，只有在第一次尝试通过网络传递数据时，才会进行真正的初始化
net.add(tf.keras.layers.Dense(1, kernel_initializer=initializer))

### 定义损失函数

In [7]:
loss = tf.keras.losses.MeanSquaredError()

### 定义优化算法

In [8]:
trainer = tf.keras.optimizers.SGD(learning_rate=0.03)

### 训练

In [9]:
num_epochs = 3
for epoch in range(num_epochs):
    for X,y in data_iter:
        with tf.GradientTape() as tape:
            l = loss(net(X, training=True), y)
        grads = tape.gradient(l, net.trainable_variables)
        trainer.apply_gradients(zip(grads, net.trainable_variables))
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000198
epoch 2, loss 0.000102
epoch 3, loss 0.000102


In [10]:
w = net.get_weights()[0]
print('w的估计误差: ', true_w - tf.reshape(w, true_w.shape))
b = net.get_weights()[1]
print('b的估计误差: ', true_b - b)

w的估计误差:  tf.Tensor([0.00019133 0.00026727], shape=(2,), dtype=float32)
b的估计误差:  [0.00067759]


In [11]:
net.get_weights()

[array([[ 1.9998087],
        [-3.4002674]], dtype=float32),
 array([4.199322], dtype=float32)]

In [12]:
help(zip)

Help on class zip in module builtins:

class zip(object)
 |  zip(*iterables) --> A zip object yielding tuples until an input is exhausted.
 |  
 |     >>> list(zip('abcdefg', range(3), range(4)))
 |     [('a', 0, 0), ('b', 1, 1), ('c', 2, 2)]
 |  
 |  The zip object yields n-length tuples, where n is the number of iterables
 |  passed as positional arguments to zip().  The i-th element in every tuple
 |  comes from the i-th iterable argument to zip().  This continues until the
 |  shortest argument is exhausted.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and ret

In [19]:
a = ("john", "Charles", "Mike")
b = ("Jenny", "Christy", "Monica", "Vicky")
x = zip(a, b)
tuple(x)

(('john', 'Jenny'), ('Charles', 'Christy'), ('Mike', 'Monica'))